# Git commands


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Master_course_Homeworks/
# تنظیم ایمیل و نام (اگر سشن جدید است)
!git config --global user.email "arshia.mokhlesi@gmail.com"
!git config --global user.name "arshia.mkh"

# ساخت برنچ برای تمرین جدید
# !git checkout -b hw6-IR

/content/drive/My Drive/Colab Notebooks/Master_course_Homeworks


In [4]:
!git add .
!git commit -m "add a code only to show the code in github\n complete LLM as judge part 2 section 6\n add report of home work "
!git push origin hw6-IR

[main fd016e9] add a code only to show the code in github\n complete LLM as judge part 2 section 6\n add report of home work
 1 file changed, 1 insertion(+), 1 deletion(-)
Everything up-to-date


In [ ]:
!git checkout main
!git pull origin main
!git merge hw6-IR
!git push origin main

error: Your local changes to the following files would be overwritten by checkout:
	Intelligence Information Retrieval/HW6/Conversational_Search_System.ipynb
Please commit your changes or stash them before you switch branches.
Aborting
From https://github.com/ArsHia-cdMstr/Master_course_Homeworks
 * branch            main       -> FETCH_HEAD
Already up to date.
Already up to date.
Everything up-to-date
